In [1]:
import torch
import numpy as np

## Baseline Settings

MWPM. We use the open-source library PyMatching42 with the noise model used for data generation as detailed in the ‘Experimentally motivated noise model’ subsection.

BP-OSD. We use the open-source library stimbposd43. We use the exact noise model used for data generation and set the maximal belief propagation iterations to 20.

MLE. We use the algorithm developed and implemented as in ref. 14.

In [2]:
syndrome = torch.load('./cached_qec_data_small/train/syndromes_d3_c3_bs1024_D1.pt')
label = torch.load('./cached_qec_data_small/train/labels_d3_c3_bs1024_D1.pt')

In [3]:
syndrome.shape, label.shape

(torch.Size([20480, 2, 2, 8]), torch.Size([20480, 2]))

The shape is (sample, timestep, qubit, detector), the batch_size is the number of shots.

In [4]:
from mccd.dataset import load_circuit_from_file

c = load_circuit_from_file('./cached_qec_data_small/train/circuits_d3_c3_bs1024_D1.txt', 0)

In [5]:
c.n_logical_qubits

2

In [6]:
from mccd.random_clifford_circuit import *

c = TypeICircuit(n_logical_qubits=1, depth=5, circuit_index='3')

In [7]:
c.sample_circuit()

In [8]:
from operator import itemgetter


sorted(c, key=itemgetter(1))

[('H', 0, [0]),
 ('H', 0, [0]),
 ('H', 1, [0]),
 ('H', 1, [0]),
 ('Z', 2, [0]),
 ('Z', 2, [0]),
 ('Z', 3, [0]),
 ('Z', 3, [0]),
 ('H', 4, [0]),
 ('H', 4, [0])]

In [9]:
c.n_logical_qubits

1

In [10]:
from operator import itemgetter
import stim


def to_stim_circuit(mccd_circuit):
    res = stim.Circuit()
    now = 0
    for n in range(mccd_circuit.n_logical_qubits):
        res.append('R', [n])

    for name, timestep, qubits in sorted(mccd_circuit, key=itemgetter(1)):
        res.append(name, qubits)
        if timestep > now:
            res.append('TICK')
            now = timestep

    for n in range(mccd_circuit.n_logical_qubits):
        res.append('M', [n])

    return res


In [11]:
circuit = to_stim_circuit(c)

In [12]:
circuit

stim.Circuit('''
    R 0
    H 0 0 0
    TICK
    H 0
    Z 0
    TICK
    Z 0 0
    TICK
    Z 0
    H 0
    TICK
    H 0
    M 0
''')

In [13]:
from surface_sim.setups import CircuitNoiseSetup
from surface_sim.models import CircuitNoiseModel
from surface_sim import Detectors
from surface_sim.experiments import schedule_from_circuit, experiment_from_schedule
from surface_sim.circuit_blocks.rot_surface_code_css import gate_to_iterator
from surface_sim.layouts import unrot_surface_codes, rot_surface_codes


layouts = rot_surface_codes(circuit.num_qubits, distance=3)
setup = CircuitNoiseSetup()
model = CircuitNoiseModel.from_layouts(setup, *layouts)
detectors = Detectors.from_layouts("pre-gate", *layouts)

setup.set_var_param("prob", 1e-3)
schedule = schedule_from_circuit(circuit, layouts, gate_to_iterator)


In [15]:
stim_circuit = experiment_from_schedule(
    schedule, model, detectors, anc_reset=True
)

TypeError: The given layout is not an unrotated surface code, but a rotated_surface_code

In [ ]:
sampler = stim_circuit.compile_detector_sampler()

In [ ]:
sampler

stim.CompiledDetectorSampler(stim.Circuit('''
    Y 0
    X 1
    I 0
    Z 1 0
    H 1
    I 0 1
    H 0
    X 1
    Z 0
    X 1 0
    H 1
    Y 0
    X 1 0
    Y 1
    I 0
    Z 1
    I 0
    Z 1
    X 0
    Y 1 0
    X 1 0
    H 1
    Z 0
    X 1
    H 0
    X 1
    I 0 1
    Z 0
    H 1
    I 0
    Z 1
    Y 0
    X 1
'''))

In [ ]:
shots, observables = sampler.sample(10, separate_observables=True)


In [ ]:
observables

array([], shape=(10, 0), dtype=bool)

In [ ]:
from mccd import CachedSyndromeDataset

dataset = CachedSyndromeDataset('./cached_qec_data_small/train/', circuit_index='3', code_distance=3, depth=3, batch_size=1024)

In [ ]:
sample = dataset[0]

In [ ]:
sample.keys()

dict_keys(['syndromes', 'label', 'final_round_syndromes', 'circuit'])

In [ ]:
sample['syndromes'].shape, sample['label'].shape

(torch.Size([1024, 6, 2, 8]), torch.Size([1024, 2]))

In [ ]:
from surface_sim.util import insert_syndrome_every
